In [2]:
import os
import pickle
import time

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import certifi
import ssl

SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = r"D:\OneDrive\OneDrive - Rowad Modern Engineering\x004 Data Science\03.rme.db\00.repo\rme.db\09.api.projects\01.youtube\02.upload.with.api\client_secret_253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com.json"

def get_authenticated_service():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            ssl_context = ssl.create_default_context(cafile=certifi.where())
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES
            )
            creds = flow.run_local_server(port=0, ssl_context=ssl_context)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('youtube', 'v3', credentials=creds)

def upload_to_youtube(video_path, title, description, tags=[], privacyStatus='unlisted'):
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description,
                "tags": tags,
                "madeForKids": False
            },
            "status": {
                "privacyStatus": privacyStatus
            }
        },
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )
    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"Uploaded {int(status.progress() * 100)}%", end="\r")
                time.sleep(0.1)
        except Exception as e:
            print(f"Error during upload: {e}")
            return
    print(f"Video uploaded successfully! Video ID: {response['id']}")

def upload_with_retry(video_path, title, description, retries=3, delay=5):
    for attempt in range(retries):
        try:
            upload_to_youtube(video_path, title, description)
            return
        except Exception as e:
            print(f"Error uploading video: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"Failed to upload video after {retries} attempts.")

def upload_videos_from_folder(folder_path):
    print(f"Looking for videos in folder: {folder_path}")
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            print(f"Found video: {filename}")
            video_path = os.path.join(folder_path, filename)
            title = filename[:-4]
            description = f"Uploaded from folder: {folder_path}"
            upload_with_retry(video_path, title, description)
        else:
            print(f"Skipping non-video file: {filename}")

folder_path = input("Enter the folder path containing the videos: ")
upload_videos_from_folder(folder_path)


Looking for videos in folder: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Acrobat\InfiniteSkills\Advanced Acrobat XI Training\02. Document Security
Found video: 02_01-Methods Of Securing A PDF.mp4
Video uploaded successfully! Video ID: Jl-pFvGW_tw
Found video: 02_02-Using Password Protection.mp4
Video uploaded successfully! Video ID: wQkuCSfT1Xw
Found video: 02_03-Using Certificate Encryption.mp4


KeyboardInterrupt: 